# Instanciar Spark

In [10]:
import pyspark.sql.functions as f
from pyspark.sql.functions import col, from_json, explode, expr
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
import pandas as pd
import os

import requests
import json

PATH = "C:\\PROJETOS\\ibge_integration\\RAW_DATA\\"  # SUBSTITUA UM PATH PARA UM DE SUA PREFERÊNCIA OU UM BUCKET
PATH = "E:\\AMBIENTE DEV\\PROJETOS\\ibge_integration\\RAW_DATA\\"  # SUBSTITUA UM PATH PARA UM DE SUA PREFERÊNCIA OU UM BUCKET

# Configuração da Spark Session
spark_session = SparkSession.builder \
    .appName("Ibge") \
    .config("spark.master", "local[*]") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .config("spark.executor.instances", "3") \
    .getOrCreate()


# SQL CONNECTION

In [11]:
# Configurações de conexão com o PostgreSQL
# Este item é totalmente editavel, para uma conexão de sua preferencia (Vamos fazer conforme o docker-compose que subimos)
url_database = "jdbc:postgresql://localhost:8085/bronze_data"
properties = {"user": "ibge", "password": "ibge", "driver": "org.postgresql.Driver"}

# engine = create_engine(url_database)

# BRONZE DATA - (Região, Estado e Municipio)

In [12]:
# LER O PARQUE DE REGIÕES E GRAVAR NO BANCO

# Ler o arquivo Parquet de regiões
df_regions = spark_session.read.parquet(f"{PATH}\\regioes.parquet")

# Renomear as colunas para adequar aos nomes do banco de dados
df_regions = df_regions.withColumnRenamed("id", "id_regiao").withColumnRenamed("nome", "regiao")

# Escrever os dados no PostgreSQL usando SQL Alchemy e JDBC
df_regions.write \
    .jdbc(url=url_database, table="regiao", mode="overwrite", properties=properties)


In [13]:
# LER O PARQUE DE ESTADO E GRAVAR NO BANCO

# Ler o arquivo Parquet de estados
df_state = spark_session.read.parquet(f"{PATH}\\estados.parquet")

# Renomear as colunas para adequar aos nomes do banco de dados
df_state = df_state.withColumnRenamed("id", "id_estado").withColumnRenamed("sigla", "uf").withColumnRenamed("nome", "estado")

# Selecionar as colunas necessárias e renomear a coluna da região
df_state_select = df_state.select(
    'id_estado',
    'uf',
    'estado',
    col('regiao.id').alias('id_regiao')
)

# Escrever os dados no PostgreSQL
df_state_select.write \
    .jdbc(url=url_database, table="estado", mode="overwrite", properties=properties)


In [14]:
# LER O PARQUE DE MUNICÍPIO E GRAVAR NO BANCO

# Ler o arquivo Parquet de municípios
df_substate = spark_session.read.parquet(f"{PATH}\\municipios.parquet")

# Renomear as colunas para adequar aos nomes do banco de dados
df_substate = df_substate.withColumnRenamed("ibge", "id_municipio").withColumnRenamed("nome", "municipio")

# Escrever os dados no PostgreSQL
df_substate.write \
    .jdbc(url=url_database, table="municipios", mode="overwrite", properties=properties)


In [15]:
# LER UM CSV COM AS CAPITAIS

# Ler o arquivo CSV das capitais
df_capital = spark_session.read.csv(f"{PATH}\\capital.csv", header=True, inferSchema=True, sep=";", encoding="ISO-8859-1")

# Selecionar as colunas relevantes e renomeá-las
df_capital = df_capital.select(
    col("Estados").alias("estado"),
    col("Capital").alias("capital"),
    col("Sigla").alias("uf")
)

# Escrever os dados no PostgreSQL
df_capital.write \
    .jdbc(url=url_database, table="capitais", mode="overwrite", properties=properties)


# PESQUISAS

In [16]:
# LER A PESQUISA IDH

# Define o subdiretório e o caminho da pasta
subfolder = 'idh'
pathstring = f'{PATH}\\{subfolder}\\'

# Lista todos os arquivos Parquet na pasta
arquivos_parquet = [os.path.join(pathstring, arquivo) for arquivo in os.listdir(pathstring) if arquivo.endswith(".parquet")]

# Lê todos os arquivos Parquet e cria um DataFrame único
df_idh = spark_session.read.parquet(*arquivos_parquet)

# Exibe a contagem de linhas no DataFrame
print(df_idh.count())

# Escreve os dados no PostgreSQL 
df_idh.write \
    .jdbc(url=url_database, table="pesquisas_idh", mode="overwrite", properties=properties)


351


In [17]:
# LER A PESQUISA PIB

# Define o subdiretório e o caminho da pasta
subfolder = 'pib'
pathstring = f'{PATH}\\{subfolder}\\'

# Lista todos os arquivos Parquet na pasta
arquivos_parquet = [os.path.join(pathstring, arquivo) for arquivo in os.listdir(pathstring) if arquivo.endswith(".parquet")]

# Lê todos os arquivos Parquet e cria um DataFrame único
df_pib = spark_session.read.parquet(*arquivos_parquet)

# Exibe a contagem de linhas no DataFrame
print(df_pib.count())

# Escreve os dados no PostgreSQL 
df_pib.write \
    .jdbc(url=url_database, table="pesquisas_pib", mode="overwrite", properties=properties)


22280


In [18]:
# LER A PESQUISA POPULAÇÃO

# Define o subdiretório e o caminho da pasta
subfolder = 'populacao'
pathstring = f'{PATH}\\{subfolder}\\'

# Lista todos os arquivos Parquet na pasta
arquivos_parquet = [os.path.join(pathstring, arquivo) for arquivo in os.listdir(pathstring) if arquivo.endswith(".parquet")]

# Lê todos os arquivos Parquet e cria um DataFrame único
df_populacao = spark_session.read.parquet(*arquivos_parquet)

# Exibe a contagem de linhas no DataFrame
print(df_populacao.count())

# Escreve os dados no PostgreSQL 
df_populacao.write \
    .jdbc(url=url_database, table="pesquisas_populacao", mode="overwrite", properties=properties)


22280
